In [1]:
from datasets import BinCountOnes
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = BinCountOnes(num_points=1200, num_classes=4)

In [3]:
data = BinCountOnes(train_size=1000, val_size=100, test_size=100)

In [3]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], data.train_data.dataset.tensors[1][:10]

(tensor([[14,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
         [ 1,  1,  1,  1,  1,  1, 18, 14,  1, 12,  1,  1],
         [11,  5, 12,  4,  1, 13, 18,  2, 11, 10, 16,  6],
         [ 2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
         [ 8,  1, 14,  5,  2, 19, 17,  1,  1, 17,  3,  7],
         [ 1,  3, 12,  1, 10,  8,  1,  1, 19,  8,  1, 10],
         [ 1,  1,  1,  1,  1,  1,  1,  1,  7,  1,  1, 15],
         [13, 18,  1,  1, 17,  2,  1,  2, 12, 16,  3,  1],
         [ 1,  1,  4,  2,  1, 15,  1,  1,  4,  1,  1, 10],
         [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]]),
 tensor([3, 2, 0, 3, 0, 1, 3, 1, 2, 3]))

In [5]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 1    504
0    496
dtype: int64
Val data:
 0    55
1    45
dtype: int64
Test data:
 1    51
0    49
dtype: int64


In [6]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json", num_labels=4)
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:181: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [7]:
np.log(2)

0.6931471805599453

In [8]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 1.382622241973877, grad: 0.0),
 node(name: +, val: [[-9.30276350e-04  3.94674158e-03  6.10539457e-03 -2.08229348e-02]
  [-9.19282145e-04  3.93936038e-03  6.08501770e-03 -2.08074003e-02]
  [-6.84643583e-03  8.59113716e-05 -1.26382541e-02 -6.37005724e-04]
  [-9.07362148e-04  3.93580412e-03  6.06687739e-03 -2.07845923e-02]
  [ 3.06057557e-03  1.99109167e-02 -1.15268566e-02 -5.82206587e-04]
  [-6.86382689e-03  9.72126945e-05 -1.26444707e-02 -6.70881302e-04]
  [-8.95297097e-04  3.99949355e-03  6.12078700e-03 -2.08104998e-02]
  [ 1.52712443e-03 -2.12904289e-02 -6.14130450e-03  1.42248522e-04]
  [-9.04530403e-04  3.95513326e-03  6.10377360e-03 -2.08220109e-02]
  [-1.51431619e-03 -3.68165155e-03  5.76670514e-03  1.04310503e-02]
  [-9.14073258e-04  3.98704037e-03  6.12411276e-03 -2.08017565e-02]
  [-9.07771231e-04  3.95555096e-03  6.06934307e-03 -2.07854062e-02]
  [-9.05169058e-04  3.94736137e-03  6.08345959e-03 -2.08019093e-02]
  [ 3.74852656e-03  3.44430865e-03  9.00856592